# DibParser (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
ls ~/.nuget/packages/argu

6.2.2


In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.2.2/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/fparsec/2.0.0-beta2/lib/netstandard2.1/FParsec.dll"
#r @"../../../../../../../.nuget/packages/fparsec/2.0.0-beta2/lib/netstandard2.1/FParsecCS.dll"

In [ ]:
#!import ../../lib/spiral/common.fsx
#!import ../../lib/spiral/sm.fsx
#!import ../../lib/spiral/date_time.fsx
#!import ../../lib/spiral/file_system.fsx
#!import ../../lib/spiral/lib.fsx
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
open Common
open FParsec

## escapeCell (test)

In [ ]:
//// test

let inline escapeCell input =
    input
    |> Sm.split "\n"
    |> Array.map (function
        | line when line |> Sm.starts_with "\\#!" || line |> Sm.starts_with "\\#r" ->
            System.Text.RegularExpressions.Regex.Replace (line, "^\\\\#", "#")
        | line -> line
    )
    |> Sm.concat "\n"

In [ ]:
//// test

$"a{nl}\\#!magic{nl}b{nl}"
|> escapeCell
|> _assertEqual (
    $"a{nl}#!magic{nl}b{nl}"
)

a
#!magic
b



## magicMarker

In [ ]:
let magicMarker : Parser<string, unit> = pstring "#!"

In [ ]:
//// test

"#!magic"
|> run magicMarker
|> _assertEqual (
    Success ("#!", (), Position ("", 2, 1, 3))
)

Success
      Item1: #!
      Item2: <null>
      Item3: Position
        Index: 2
        Line: 1
        Column: 3
        StreamName: 


In [ ]:
//// test

"##!magic"
|> run magicMarker
|> _assertEqual (
    Failure (
        $"Error in Ln: 1 Col: 1{nl}##!magic{nl}^{nl}Expecting: '#!'{nl}",
        ParserError (
            Position ("", 0, 1, 1),
            (),
            ErrorMessageList (ExpectedString "#!")
        ),
        ()
    )
)

Failure
      Item1: Error in Ln: 1 Col: 1
##!magic
^
Expecting: '#!'

      Item2: ParserError
        Position: Position
          Index: 0
          Line: 1
          Column: 1
          StreamName: 
        UserState: <null>
        Messages: ErrorMessageList
          Head: ExpectedString
            String: #!
            Type: ExpectedString
          Tail: <null>
      Item3: <null>


## magicCommand

In [ ]:
let magicCommand =
    magicMarker
    >>. manyTill anyChar newline
    |>> (System.String.Concat >> Sm.trim)

In [ ]:
//// test

"#!magic

a"
|> run magicCommand
|> _assertEqual (
    Success ("magic", (), Position ("", 8, 2, 1))
)

Success
      Item1: magic
      Item2: <null>
      Item3: Position
        Index: 8
        Line: 2
        Column: 1
        StreamName: 


In [ ]:
//// test

" #!magic

a"
|> run magicCommand
|> _assertEqual (
    Failure (
        $"Error in Ln: 1 Col: 1{nl} #!magic{nl}^{nl}Expecting: '#!'{nl}",
        ParserError (
            Position ("", 0, 1, 1),
            (),
            ErrorMessageList (ExpectedString "#!")
        ),
        ()
    )
)

Failure
      Item1: Error in Ln: 1 Col: 1
 #!magic
^
Expecting: '#!'

      Item2: ParserError
        Position: Position
          Index: 0
          Line: 1
          Column: 1
          StreamName: 
        UserState: <null>
        Messages: ErrorMessageList
          Head: ExpectedString
            String: #!
            Type: ExpectedString
          Tail: <null>
      Item3: <null>


## content

In [ ]:
let content =
    (newline >>. magicMarker) <|> (eof >>. preturn "")
    |> attempt
    |> lookAhead
    |> manyTill anyChar
    |>> (System.String.Concat >> Sm.trim)

In [ ]:
//// test

"#!magic


a


"
|> run content
|> _assertEqual (
    Success ("#!magic


a", (), Position ("", 14, 7, 1))
)

Success
      Item1: #!magic


a
      Item2: <null>
      Item3: Position
        Index: 14
        Line: 7
        Column: 1
        StreamName: 


## Block

In [ ]:
type Block =
    {
        magic : string
        content : string
    }

## block

In [ ]:
let block =
    pipe2
        magicCommand
        content
        (fun magic content ->
            {
                magic = magic
                content = content
            })

In [ ]:
//// test

"#!magic


a


"
|> run block
|> _assertEqual (
    Success (
        { magic = "magic"; content = "a" },
        (),
        Position ("", 14, 7, 1)
    )
)

Success
      Item1: Block
        magic: magic
        content: a
      Item2: <null>
      Item3: Position
        Index: 14
        Line: 7
        Column: 1
        StreamName: 


## blocks

In [ ]:
let blocks =
    skipMany newline
    >>. sepEndBy block (skipMany1 newline)

In [ ]:
//// test


"#!magic1

a

\#!magic2

b

"
|> escapeCell
|> run blocks
|> _assertEqual (
    Success (
        [
            { magic = "magic1"; content = "a" }
            { magic = "magic2"; content = "b" }
        ],
        (),
        Position ("", 26, 9, 1)
    )
)

Success
      Item1: FSharpList<Block>
        - magic: magic1
          content: a
        - magic: magic2
          content: b
      Item2: <null>
      Item3: Position
        Index: 26
        Line: 9
        Column: 1
        StreamName: 


## Output

In [ ]:
type Output =
    | Fs
    | Md
    | Spi
    | Spir

let inline kernelOutputs magic =
    match magic with
    | "fsharp" -> [ Fs ]
    | "markdown" -> [ Md ]
    | "spiral" -> [ Spi; Spir ]
    | _ -> []

## formatBlock

In [ ]:
let inline formatBlock output (block : Block) =
    match output, block with
    | output, { magic = "markdown"; content = content } ->
        let markdownComment =
            match output with
            | Spi | Spir -> "// // "
            | Fs -> "/// "
            | _ -> ""
        content
        |> Sm.split "\n"
        |> Array.map (Sm.trim_end [||])
        |> Array.filter (Sm.ends_with " (test)" >> not)
        |> Array.map (function
            | "" -> markdownComment |> Sm.trim
            | line -> System.Text.RegularExpressions.Regex.Replace (line, "^\\s*", $"$&{markdownComment}")
        )
        |> Sm.concat "\n"
    | Fs, { magic = "fsharp"; content = content } ->
        let trimmedContent = content |> Sm.trim
        if trimmedContent |> Sm.starts_with "//// test" || trimmedContent |> Sm.starts_with "//// ignore"
        then ""
        else
            content
            |> Sm.split "\n"
            |> Array.filter (Sm.trim_start [||] >> Sm.starts_with "#r" >> not)
            |> Sm.concat "\n"
    | (Spi | Spir), { magic = "spiral"; content = content } ->
        let trimmedContent = content |> Sm.trim
        if trimmedContent |> Sm.starts_with "// // test" || trimmedContent |> Sm.starts_with "// // ignore"
        then ""
        else content
    | _ -> ""

In [ ]:
//// test

"#!markdown


a

    b

c


\#!markdown


c


\#!fsharp


let a = 1"
|> escapeCell
|> run block
|> function
    | Success (block, _, _) -> formatBlock Fs block
    | Failure (msg, _, _) -> failwith msg
|> _assertEqual "/// a
///
    /// b
///
/// c"

/// a
///
    /// b
///
/// c


## formatBlocks

In [ ]:
let inline formatBlocks output blocks =
    blocks
    |> List.map (formatBlock output)
    |> List.filter ((<>) "")
    |> Sm.concat "\n\n"
    |> fun s -> s + "\n"

In [ ]:
//// test

"#!markdown


a

b


\#!markdown


c


\#!fsharp


let a = 1

\#!markdown

d (test)

\#!fsharp

//// test

let a = 2

\#!markdown

e

\#!fsharp

let a = 3"
|> escapeCell
|> run blocks
|> function
    | Success (blocks, _, _) -> formatBlocks Fs blocks
    | Failure (msg, _, _) -> failwith msg
|> _assertEqual "/// a
///
/// b

/// c

let a = 1

/// e

let a = 3
"

/// a
///
/// b

/// c

let a = 1

/// e

let a = 3



## parse

In [ ]:
let inline parse output input =
    match run blocks input with
    | Success (blocks, _, _) ->
        let blocks =
            blocks
            |> List.filter (fun block ->
                block.magic |> kernelOutputs |> List.contains output || block.magic = "markdown"
            )

        match blocks with
        | { magic = "markdown"; content = content } :: _
            when output = Fs
            && content |> Sm.starts_with "# "
            && content |> Sm.ends_with ")"
            ->
            let inline indentBlock (block : Block) =
                { block with
                    content =
                        block.content
                        |> Sm.split "\n"
                        |> Array.fold
                            (fun (lines, isMultiline) line ->
                                let trimmedLine = line |> Sm.trim
                                if trimmedLine = ""
                                then "" :: lines, isMultiline
                                else
                                    let inline singleQuoteLine () =
                                        trimmedLine |> Seq.sumBy ((=) '"' >> System.Convert.ToInt32) = 1
                                        && trimmedLine |> Sm.contains @"'""'" |> not
                                        && trimmedLine |> Sm.ends_with "{" |> not
                                        && trimmedLine |> Sm.ends_with "{|" |> not
                                        && trimmedLine |> Sm.starts_with "}" |> not
                                        && trimmedLine |> Sm.starts_with "|}" |> not

                                    match isMultiline, trimmedLine |> Sm.split_string [| $"{q}{q}{q}" |] with
                                    | false, [| _; _ |] ->
                                        $"    {line}" :: lines, true

                                    | true, [| _; _ |] ->
                                        line :: lines, false

                                    | false, _ when singleQuoteLine () ->
                                        $"    {line}" :: lines, true

                                    | false, _ when line |> Sm.starts_with "#" && block.magic = "fsharp" ->
                                        line :: lines, false

                                    | false, _ ->
                                        $"    {line}" :: lines, false

                                    | true, _ when singleQuoteLine () && line |> Sm.starts_with "    " ->
                                        $"    {line}" :: lines, false

                                    | true, _ when singleQuoteLine () ->
                                        line :: lines, false

                                    | true, _ ->
                                        line :: lines, true
                            )
                            ([], false)
                        |> fst
                        |> List.rev
                        |> Sm.concat "\n"
                }

            let moduleName, namespaceName =
                System.Text.RegularExpressions.Regex.Match (content, @"# (.*) \((.*)\)$")
                |> fun m -> m.Groups.[1].Value, m.Groups.[2].Value

            let moduleBlock =
                {
                    magic = "fsharp"
                    content =
                        $"#if !INTERACTIVE
namespace {namespaceName}
#endif

module {moduleName} ="
                }

            blocks
            |> List.indexed
            |> List.fold
                (fun blocks (index, block) ->
                    match index with
                    | 0 -> blocks
                    | 1 -> indentBlock block :: moduleBlock :: blocks
                    | _ -> indentBlock block :: blocks
                )
                []
            |> List.rev
        | _ -> blocks
        |> Result.Ok
    | Failure (errorMsg, _, _) -> Result.Error errorMsg

In [ ]:
//// test

let example1 =
    $"""#!meta

{{"kernelInfo":{{"defaultKernelName":"fsharp","items":[{{"aliases":[],"name":"fsharp"}},{{"aliases":[],"name":"fsharp"}}]}}}}

\#!markdown

# TestModule (TestNamespace)

\#!fsharp

\#!import file.dib

\#!fsharp

\#r "nuget:Expecto"

\#!markdown

## ParserLibrary

\#!fsharp

open System

\#!markdown

## x (test)

\#!fsharp

//// ignore

let x = 1

\#!spiral

// // test

inl x = 0i32

\#!spiral

inl x = 0i32

\#!markdown

### TextInput

\#!fsharp

let str1 = "abc
def"

let str2 =
    "abc\
def"

let str3 =
    $"1{{
        1
    }}1"

let str4 =
    $"1{{({{|
        a = 1
    |}}).a}}1"

let str5 =
    "abc \
        def"

let x =
    match '"' with
    | '"' -> true
    | _ -> false

let long1 = {q}{q}{q}a{q}{q}{q}

let long2 =
    {q}{q}{q}
a
{q}{q}{q}

\#!fsharp

type Position =
    {{
#if INTERACTIVE
        line : string
#else
        line : int
#endif
        column : int
    }}"""
    |> escapeCell

In [ ]:
//// test

example1
|> parse Fs
|> Result.toOption
|> Option.get
|> (formatBlocks Fs)
|> _assertEqual $"""#if !INTERACTIVE
namespace TestNamespace
#endif

module TestModule =

    /// ## ParserLibrary

    open System

    /// ### TextInput

    let str1 = "abc
def"

    let str2 =
        "abc\
def"

    let str3 =
        $"1{{
            1
        }}1"

    let str4 =
        $"1{{({{|
            a = 1
        |}}).a}}1"

    let str5 =
        "abc \
            def"

    let x =
        match '"' with
        | '"' -> true
        | _ -> false

    let long1 = {q}{q}{q}a{q}{q}{q}

    let long2 =
        {q}{q}{q}
a
{q}{q}{q}

    type Position =
        {{
#if INTERACTIVE
            line : string
#else
            line : int
#endif
            column : int
        }}
"""

#if !INTERACTIVE
namespace TestNamespace
#endif

module TestModule =

    /// ## ParserLibrary

    open System

    /// ### TextInput

    let str1 = "abc
def"

    let str2 =
        "abc\
def"

    let str3 =
        $"1{
            1
        }1"

    let str4 =
        $"1{({|
            a = 1
        |}).a}1"

    let str5 =
        "abc \
            def"

    let x =
        match '"' with
        | '"' -> true
        | _ -> false

    let long1 = """a"""

    let long2 =
        """
a
"""

    type Position =
        {
#if INTERACTIVE
            line : string
#else
            line : int
#endif
            column : int
        }



In [ ]:
//// test

example1
|> parse Md
|> Result.toOption
|> Option.get
|> (formatBlocks Md)
|> _assertEqual "# TestModule (TestNamespace)

## ParserLibrary

### TextInput
"

# TestModule (TestNamespace)

## ParserLibrary

### TextInput



In [ ]:
//// test

example1
|> parse Spi
|> Result.toOption
|> Option.get
|> (formatBlocks Spi)
|> _assertEqual "// // # TestModule (TestNamespace)

// // ## ParserLibrary

inl x = 0i32

// // ### TextInput
"

// // # TestModule (TestNamespace)

// // ## ParserLibrary

inl x = 0i32

// // ### TextInput



## parseDibCode

In [ ]:
let inline parseDibCode output file = async {
    let getLocals () = $"output: {output} / file: {file} / {getLocals ()}"
    trace Debug (fun () -> "parseDibCode") getLocals
    let! input = file |> FileSystem.readAllTextAsync
    match parse output input with
    | Result.Ok blocks -> return blocks |> formatBlocks output
    | Result.Error msg -> return failwith msg
}

## writeDibCode

In [ ]:
let inline writeDibCode output path = async {
    let getLocals () = $"output: {output} / path: {path} / {getLocals ()}"
    trace Debug (fun () -> "writeDibCode") getLocals
    let! result = parseDibCode output path
    let outputPath = path |> Sm.replace ".dib" $".{output |> string |> Sm.to_lower}"
    do! result |> FileSystem.writeAllTextAsync outputPath
}

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.MainCommand; Argu.ArguAttributes.Mandatory>]
        File of file : string * Output

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | File _ -> nameof File

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

USAGE: dotnet-repl [--help] <file> <fs|md|spi|spir>

FILE:

    <file> <fs|md|spi|spir>
                          File

OPTIONS:

    --help                display this list of options.


## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let files =
        argsMap.[nameof Arguments.File]
        |> List.map (function
            | Arguments.File (path, output) -> path, output
        )

    files
    |> List.map (fun (path, output) -> path |> writeDibCode output)
    |> Async.Parallel
    |> Async.Ignore
    |> Async.runWithTimeout 30000
    |> function
        | Some () -> 0
        | None -> 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0

00:00:00 #1 [Debug] writeDibCode / output: Fs / path: DibParser.dib
00:00:00 #2 [Debug] parseDibCode / output: Fs / file: DibParser.dib
